In [9]:
import pandas as pd

# Load the data from CSV files
choices_df = pd.read_csv('data/choices_table.csv')
bids_df = pd.read_csv('data/bids_table.csv')
seniority_df = pd.read_csv('data/seniority_table.csv')
employee_df = pd.read_csv('data/employee_table.csv')

# Step 1: Create a mapping from employee number to seniority
seniority_mapping = dict(zip(seniority_df['employee_number'], seniority_df['seniority_number']))

# Step 2: Sort employees by seniority number
bids_df['seniority_number'] = bids_df['employee_number'].map(seniority_mapping)
bids_df = bids_df.sort_values(by='seniority_number')

# Step 3: Create a dictionary to store the assigned choices
assigned_choices = {}

# Step 4: Loop over each employee based on their seniority
for index, row in bids_df.iterrows():
    employee_number = row['employee_number']
    # Get the list of choices for the employee (from 1st to 21st)
    choices = row[2:].dropna().values.astype(int)
    
    # Try to assign the employee to their preferred choice
    assigned = False
    for choice in choices:
        # Check if the choice is available (i.e., hasn't been assigned yet)
        if choice not in assigned_choices.values():
            assigned_choices[employee_number] = choice
            assigned = True
            break
        
    # If no choice is available, the employee remains unassigned (though it should not happen)
    if not assigned:
        assigned_choices[employee_number] = None

# Step 5: Merge the results with employee details to get first_name and last_name
result = pd.DataFrame(list(assigned_choices.items()), columns=['employee_number', 'choice'])
result = result.merge(employee_df[['employee_number', 'first_name', 'last_name']], on='employee_number')

# Step 6: Map the choice number to the actual shifts for each day (Monday to Sunday)
def get_shift_days(choice_number):
    # Get the row from choices_df that corresponds to this choice_number
    shift_row = choices_df.iloc[choice_number - 1, 1:].values  # Get Monday-Sunday shifts (excluding row_number)
    return shift_row

# Get the shifts for each day of the week by using apply and assigning to each column separately
shift_columns = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
shift_values = result['choice'].map(lambda x: get_shift_days(x))

# Convert the list of shifts to individual columns in the DataFrame
for i, day in enumerate(shift_columns):
    result[day] = shift_values.apply(lambda x: x[i])

# Step 7: Drop the 'choice' column and print or save the result
result = result.drop(columns=['choice'])
print(result)


   employee_number first_name  last_name Monday Tuesday Wednesday Thursday  \
0            E4711    Jessica      Davis     MO      MO        MO       MO   
1            E1531      James     Miller     MO      MO        MO       MO   
2            E7295      Emily      Brown    Off     Off        MO       MO   
3            E6692    Michael   Williams     MO     Off       Off       MO   
4            E2177     Andrew     Thomas     MO      MO       Off      Off   
5            E5063      Sarah     Garcia    Off      MO        MO       MO   
6            E9298    Matthew     Wilson     MO      MO        MO      Off   
7            E6966      Laura  Hernandez     SW      SW        SW       SW   
8            E3916     Robert   Martinez    Off      GY        GY       GY   
9            E5002       John      Smith     GY     Off       Off       GY   
10           E2174     Daniel      Lopez     SW      SW        SW      Off   
11           E5938     Olivia     Taylor    Off     Off        S

In [10]:
# Save the result as a CSV file
result.to_csv('data/shift_awards.csv', index=False)


In [11]:
from IPython.display import display, HTML

# Set CSS to prevent wrapping for tables
display(HTML("<style>.dataframe td { white-space: nowrap; }</style>"))

display(HTML(result.to_html(max_cols=None, max_rows=None)))


,employee_number,first_name,last_name,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,E4711,Jessica,Davis,MO,MO,MO,MO,MO,Off,Off
1,E1531,James,Miller,MO,MO,MO,MO,Off,Off,MO
2,E7295,Emily,Brown,Off,Off,MO,MO,MO,MO,MO
3,E6692,Michael,Williams,MO,Off,Off,MO,MO,MO,MO
4,E2177,Andrew,Thomas,MO,MO,Off,Off,MO,MO,MO
5,E5063,Sarah,Garcia,Off,MO,MO,MO,MO,MO,Off
6,E9298,Matthew,Wilson,MO,MO,MO,Off,Off,MO,MO
7,E6966,Laura,Hernandez,SW,SW,SW,SW,SW,Off,Off
8,E3916,Robert,Martinez,Off,GY,GY,GY,GY,GY,Off
9,E5002,John,Smith,GY,Off,Off,GY,GY,GY,GY


# -------------DONE--------------